###  00-connectivity

In [91]:
''' ENV CONFIGURATION '''
# pip install python-dotenv
import os
from dotenv import load_dotenv
from pathlib import Path


dotenv_path = os.path.join(os.getcwd(), ".env")  # Replace ".env" with the actual file path if it's in a different location
load_dotenv(dotenv_path=dotenv_path, override=True)

ACCOUNT = os.environ.get("ACCOUNT", "default")
USER = os.environ.get("USER", "default")
PASSWORD = os.environ.get("PASSWORD", "default")
ROLE = os.environ.get("ROLE", "default")
DATABASE = os.environ.get("DATABASE", "default")
WAREHOUSE = os.environ.get("WAREHOUSE", "default")
# a3491fb2-000f-4d9f-943e-127cfe29c39c

## STEP 01 - CREATE USER AND WH 

In [92]:
from snowflake.snowpark import Session
import sys
import logging

# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')

# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters =  {
       "ACCOUNT":ACCOUNT,
        "USER":USER,
        "PASSWORD":PASSWORD,
        "ROLE":ROLE,
        "DATABASE":DATABASE,
        "SCHEMA":"ALF_SNOWPARK",
        "WAREHOUSE":WAREHOUSE,
        "session_parameters":{
            'QUERY_TAG': 'DevOPS_deployment',
            'use_openssl_only': True
        }
    }
    # print(connection_parameters)
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()   

def main():
    session = get_snowpark_session()
    
    context_df = session.sql("select current_role(), current_database(), current_schema(), current_warehouse()")
    context_df.show(2)

    customer_df = session.sql("select c_custkey,c_name,c_phone,c_mktsegment from snowflake_sample_data.tpch_sf1.customer limit 10")
    customer_df.show(5)

if __name__ == '__main__':
    main()  


01:01:58 - INFO - Snowflake Connector for Python Version: 3.7.1, Python Version: 3.8.8, Platform: Windows-10-10.0.22621-SP0
01:01:58 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
01:01:59 - INFO - Snowpark Session information: 
"version" : 1.14.0,
"python.version" : 3.8.8rc1,
"python.connector.version" : 3.7.1,
"python.connector.session.id" : 343664822094,
"os.name" : Windows

01:01:59 - INFO - query: [select current_role(), current_database(), current_schema(), current_warehouse()...]
01:01:59 - INFO - query execution done
01:01:59 - INFO - Number of results in first chunk: 0
01:01:59 - INFO - query: [SELECT  *  FROM (select current_role(), current_database(), current_schema(), cu...]
01:02:00 - INFO - query execution done
01:02:00 - INFO - Number of results in first chunk: 1
----------

## STEP 02 - CREATE SCHEMA 

## STEP 03 - CREATE INTERNAL STAGE IN SOURCE SCHEMA 

## Step 4.2 Loading Data To Internal Stage Using Snowpark File API
Following Snowpark Program is using File API to read the data from local machine and loading into Snowpark Internal Stage.

In [93]:

def traverse_directory(directory, file_extension) -> list:
	local_file_path = []
	file_name = [] # list to csv file path 
	partition_dir = []
	print(directory)
	for root, dirs, files in os.walk(directory):
		for file in files:
			if file.endswith(file_extension):
				file_path = os.path.join(root, file)
				file_name.append(file)
				partition_dir.append(root.replace(directory, ''))
				local_file_path.append(file_path)

	return file_name, partition_dir, local_file_path


def main():
	# Specify the directory path to traverse
	# Get the current working directory
	current_dir = os.getcwd()
	current_dir = os.path.abspath('..')
	# Specify the path to the "dataset/sales" directory
	directory_path = os.path.join(current_dir, 'dataset', 'sales')

	# Print the path
	# print(directory_path)

	csv_file_name, csv_partition_dir , csv_local_file_path= traverse_directory(directory_path,'.csv')

	parquet_file_name, parquet_partition_dir , parquet_local_file_path= traverse_directory(directory_path,'.parquet')
	json_file_name, json_partition_dir , json_local_file_path= traverse_directory(directory_path,'.json')
	stage_location = "@DEMO_DB.ALF_SOURCE.MY_INTERNAL_STAGE"




	csv_index = 0

	for file_element in csv_file_name:
		csv_partition_dir[csv_index] = csv_partition_dir[csv_index].replace("\\" , "/")
		put_result = (
			get_snowpark_session().file.put(
				local_file_name=csv_local_file_path[csv_index],
				stage_location=rf"{stage_location+csv_partition_dir[csv_index]}",
			auto_compress=True, overwrite=True, parallel=10)
		)
		print(file_element," => ",put_result[0].status)
		csv_index+=1

	parquet_index = 0
	for file_element in parquet_file_name:
		parquet_partition_dir[parquet_index] = parquet_partition_dir[parquet_index].replace("\\" , "/")
		put_result = ( 
			get_snowpark_session().file.put( 
				local_file_name=parquet_local_file_path[parquet_index], 
				stage_location=rf"{stage_location+parquet_partition_dir[parquet_index]}",
				auto_compress=True, overwrite=True, parallel=10)
			)
		print(file_element," => ",put_result[0].status)
		parquet_index+=1


	json_index = 0
	for file_element in parquet_file_name:
		json_partition_dir[json_index] = json_partition_dir[json_index].replace("\\" , "/")
		put_result = ( 
			get_snowpark_session().file.put( 
				json_local_file_path[json_index], 
				# stage_location+"/"+json_partition_dir[json_index], 
				stage_location=rf"{stage_location+json_partition_dir[json_index]}",
				auto_compress=True, overwrite=True, parallel=10)

			)
		print(file_element," => ",put_result[0].status)
		json_index+=1 




if __name__ == '__main__':
	main()



c:\Users\aseka\repos\snowflake-coding\snowpark-end2end-sample-data\dataset\sales
c:\Users\aseka\repos\snowflake-coding\snowpark-end2end-sample-data\dataset\sales
c:\Users\aseka\repos\snowflake-coding\snowpark-end2end-sample-data\dataset\sales
01:02:00 - INFO - Snowflake Connector for Python Version: 3.7.1, Python Version: 3.8.8, Platform: Windows-10-10.0.22621-SP0
01:02:00 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
01:02:01 - INFO - Snowpark Session information: 
"version" : 1.14.0,
"python.version" : 3.8.8rc1,
"python.connector.version" : 3.7.1,
"python.connector.session.id" : 343664819094,
"os.name" : Windows

01:02:01 - INFO - query: [PUT 'file://c:/Users/aseka/repos/snowflake-coding/snowpark-end2end-sample-data/d...]
01:02:01 - INFO - query execution done
order-20200101.csv  =>  U

## STEP 05: FILE FORMATS IN COMMON SCHEMA

## Step-5.1 Select Statements On Internal Stage (CSV, Parquet, JSON)


## STEP - 6: FOREX DATA 

## Step-7.1 Loading Data From Internal Stage to Source Tables


## 7.2 Source Table DDL Script
